In this project, you will begin working with COBRA (COnstraint-Based Reconstruction and Analysis) methods using the Python library [cobrapy](https://opencobra.github.io/cobrapy/). Originally implemented in MATLAB as the COBRA Toolbox, cobrapy is an object-oriented programming framework for the reconstruction and analysis of constraint-based metabolic models. Although still not as comprehensive as the COBRA Toolbox when it comes to modeling capabilities and implemented algorithms, it provides ample functionality to cover our needs for this course. Before starting with this project, I strongly encourage you to take a look at the [documentation](https://cobrapy.readthedocs.io/en/latest) and [API reference](https://cobrapy.readthedocs.io/en/latest/autoapi/index.html) where you will find useful descriptions and minor tutorials on how to work with the library. We will also be using the web-based tool [Escher](https://escher.github.io/#/}) for the visualization of metabolic pathway maps and calculated flux phenotypes.


In [152]:
# Imports:
import cobra as c
import json


### 1.1 Toy model

Consider the following toy metabolic network with 8 reactions ($\mathrm{R}_1$, ..., $\mathrm{R}_8$) and 8 metabolites ($\mathrm{A}$, $\mathrm{B}$, $\mathrm{C}$, $\mathrm{D}$, $\mathrm{E}$, $\mathrm{ATP}$, $\mathrm{ADP}$, and $\mathrm{P}_\mathrm{i}$) (see supplementary pdf file).


In [153]:
### Define model: ---
model = c.Model("Toy model")

### Metabolites: ---
A = c.Metabolite("A")
B = c.Metabolite("B")
C = c.Metabolite("C")
D = c.Metabolite("D")
E = c.Metabolite("E")
ATP = c.Metabolite("ATP")
ADP = c.Metabolite("ADP")
P_i = c.Metabolite("P_i")


### Reactions: ---
R1 = c.Reaction("R1")
R1.add_metabolites({
   A : 1.0
})

R2 = c.Reaction("R2")
R2.add_metabolites({
   A : -1.0,
   ADP : -1.0,
   P_i : -1.0,
   B : 1.0,
   C : 1.0,
   ATP : 1.0
})

R3 = c.Reaction("R3")
R3.add_metabolites({
   C : -1.0,
   ATP : -1.0,
   B : 1.0,
   ADP : 1.0,
   P_i : 1.0
})

R4 = c.Reaction("R4")
R4.add_metabolites({
   B : -1.0,
   D : 1.0
})

R5 = c.Reaction("R5")
R5.add_metabolites({
   A : -1.0,
   D : -1.0,
   ADP : -1.0,
   P_i : -1.0,
   E : 1.0,
   ATP : 1.0
})

R6 = c.Reaction("R6")
R6.add_metabolites({
   E : -1.0,
   C : 1.0
})

R7 = c.Reaction("R7")
R7.add_metabolites({
   ATP : -1.0,
   ADP : 1.0,
   P_i : 1.0
})

R8 = c.Reaction("R8")
R8.add_metabolites({
   D : -1.0
})


### Flux bound: ----
R1.upper_bound = 10  # unit: mmol g DW^(-1)h^(-1)

### Add reactions to model: ---
model.add_reactions([R1, R2, R3, R4, R5, R6, R7, R8])


(i) Specify which metabolites are transported across the cell boundaries (e.g., cell membrane), and the direction of transport.

<span style="color:blue">
Just based on the reactions as displayed in the task description I would say that metabolite A is transported in accros the cell boundry in R1, and metabolite D is transported out accros the same boundry in R8

(ii) Write down the stoichiometric matrix **S** using the ordering of reactions and metabolites as defined above. How many degrees of freedom does this reaction system have and what is the dimensionality of the solution space (i.e. null space of **S**)?

<span style="color:blue">
The reaction system has __ degrees of freedom. The dimensionality of the solution space is __.

In [154]:
### Print the reactions: ---
print("Reactions:")
for reaction in model.reactions:
    print(f"{reaction.id} : {reaction.reaction}")

print("\n\nStoichiometric matrix: \n", c.util.create_stoichiometric_matrix(model))

Reactions:
R1 :  --> A
R2 : A + ADP + P_i --> ATP + B + C
R3 : ATP + C --> ADP + B + P_i
R4 : B --> D
R5 : A + ADP + D + P_i --> ATP + E
R6 : E --> C
R7 : ATP --> ADP + P_i
R8 : D --> 


Stoichiometric matrix: 
 [[ 1. -1.  0.  0. -1.  0.  0.  0.]
 [ 0. -1.  1.  0. -1.  0.  1.  0.]
 [ 0. -1.  1.  0. -1.  0.  1.  0.]
 [ 0.  1.  1. -1.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  1.  0.  0.]
 [ 0.  1. -1.  0.  1.  0. -1.  0.]
 [ 0.  0.  0.  1. -1.  0.  0. -1.]
 [ 0.  0.  0.  0.  1. -1.  0.  0.]]


(iii) Given an upper flux bound for $\mathrm{R}_1$ of 10 mmol gDW<sup>-1</sup> h<sup>-1</sup>, what is the maximal attainable flux through reaction $\mathrm{R}_8$ and the corresponding flux distribution? What is the net production of ATP (i.e., the flux through $\mathrm{R}_7$)? Implement the model using cobrapy and verify your answer by selecting $\mathrm{R}_8$ as the objective and maximizing its flux.

The `objective` is an attribute of the Model object, while the lower and upper flux bounds of a reaction is given by the attributes `lower_bound` and `upper_bound` of the corresponding Reaction object, respectively.

In [155]:
# Maximise the flux through R8:
model.objective = "R8"
#print(model.objective)

# Display the flux distribution:
solution = model.optimize()
print(solution.fluxes)

R1    10.0
R2    10.0
R3    10.0
R4    20.0
R5     0.0
R6     0.0
R7     0.0
R8    20.0
Name: fluxes, dtype: float64


<span style="color:blue">
The maximal attainable flux through R8 is 20 mmol gDW<sup>-1</sup> h<sup>-1</sup> when the flux through R1 is set to 10 mmol gDW<sup>-1</sup> h<sup>-1</sup>. I am not sure if the wording of the question is correct, given that ATP is not produced in R7. Either way, there will be no net production of ATP in this cascade because all the (10) ATP that is produced in R2 is furthere consumed in R3. The reactions R5, R6, and R7 are not initialized when we want to maximise the flux trhough R8 because all of the A is already concumed in R2. So, there is no ATP for R7.

(iv) It has been shown that the maximization of ATP yield in certain instances is a realistic cellular objective. Given the same flux bound for $\mathrm{R}_1$ as in (iii), explain and discuss the maximal feasible net production of ATP. Verify your answer using cobrapy.

In [156]:
model.objective = "R7"
print(model.optimize().fluxes)

R1    10.0
R2    10.0
R3    10.0
R4    20.0
R5     0.0
R6     0.0
R7     0.0
R8    20.0
Name: fluxes, dtype: float64


<span style="color:blue">
NB! The code seems to be stuck at optimizing for R8. Either way 

(v) Assume that the flux of reaction $\mathrm{R}_6$ is to be constrained to zero. You may implement this in the stoichiometric matrix by adding a new row to **S** where all column entries are zero except in column 6, which is 1. Explain why this will constrain the flux of reaction $\mathrm{R}_6$ to zero. What is the dimensionality of this new stoichiometric matrix?

### 1.2 *Escherichia coli* core model

(i) Download `ecoli_core_model` from Blackboard and read the model using the `read_sbml_model` function in cobrapy. Give a description of its content (i.e., number of reactions, metabolites, genes, etc.). Which metabolic subsystems are implemented in the model?

Hint: the subsystems are found in the Model attribute `groups`.

In [157]:
# Generate the model:
ecoli = c.io.read_sbml_model("ecoli_core_model/ecoli_core_model.xml")

# Summary:
print("Summary of the model:")
display(ecoli)

# Subsystems:
print("The metabolic subsystems implemented in the model is:")
for i in ecoli.groups:
    print("\t", i.name)

Summary of the model:


Name,COBRAModel
Memory address,18bf21380d0
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,10
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"Cytoplasm, Extracellular"


The metabolic subsystems implemented in the model is:
	 Anaplerotic reactions
	 Citric Acid Cycle
	 Exchange
	 Glutamate Metabolism
	 Glycolysis/Gluconeogenesis
	 Inorganic Ion Transport and Metabolism
	 Oxidative Phosphorylation
	 Pentose Phosphate Pathway
	 Pyruvate Metabolism
	 Transport, Extracellular


(ii) Simulate the optimal growth phenotype on aerobic, minimal glucose media, by setting the lower bound of glucose uptake ('EX_glc__D_e') to a biologically reasonable uptake rate of -18.5 mmol gDW<sup>-1</sup> h<sup>-1</sup> and the oxygen uptake EX_o2_e to -1000 mmol gDW<sup>-1</sup> h<sup>-1</sup> <sup>i</sup>. What is the maximal specific growth rate and what are the uptake fluxes of glucose, ammonia, oxygen, and inorganic phosphate in the optimal solution?

<sup>i</sup> Note that uptake reaction flux bounds by default are negative, which is due to how these traditionally are defined in constraint-based models of metabolism. A boundary metabolite X is taken up by the system using the following format for the exchange reaction X $\Longleftrightarrow$, where a positive and negative flux denotes secretion and uptake, respectively.

In [158]:
# Set glucose uptake:
ecoli.reactions.get_by_id("EX_glc__D_e").lower_bound = -18.5
# Set oxygen uptake:
ecoli.reactions.get_by_id("EX_o2_e").lower_bound = -1000
# Get the summary:
ecoli.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc-D[e],EX_glc__D_e,18.5,6,100.00%
nh4[e],EX_nh4_e,9.014,0,0.00%
o2[e],EX_o2_e,38.74,0,0.00%
pi[e],EX_pi_e,6.081,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2[e],EX_co2_e,-40.65,1,100.00%
h2o[e],EX_h2o_e,-52.69,0,0.00%
h[e],EX_h_e,-33.16,0,0.00%


<span style="color:blue">
In the optimal solution the uptake fluxes of glucose is 18.5, ~9.0 for $NH_4^+$, ~38.7 for $O_2$, and ~6.1 for $P_i$.

(iii) What is the secretion profile of anaerobic growth on glucose? Compare with that of aerobic growth on glucose (ii) and give a biochemical explanation for their differences.

In [159]:
# At anaerobic conditions the uptake of oxygen must be 0.
# I.e. set the lower bound of reaction EX_o2_e to 0:
ecoli.reactions.get_by_id("EX_o2_e").lower_bound = 0

# Get the summary:
ecoli.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2[e],EX_co2_e,0.8408,1,0.75%
glc-D[e],EX_glc__D_e,18.5,6,99.25%
h2o[e],EX_h2o_e,12.09,0,0.00%
nh4[e],EX_nh4_e,2.566,0,0.00%
pi[e],EX_pi_e,1.731,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac[e],EX_ac_e,-15.17,2,33.05%
etoh[e],EX_etoh_e,-14.67,2,31.97%
for[e],EX_for_e,-32.12,1,34.98%
h[e],EX_h_e,-56.73,0,0.00%


<span style="color:blue">


(iv) Visualize the reaction fluxes of both the aerobic and anaerobic flux phenotypes using the *E. coli* core model pathway map (found [here](https://escher.github.io/#/app?map=e_coli_core.Core\%20metabolism&tool=Builder&model=e_coli_core)) by creating a dictionary of reaction ids and corresponding fluxes, then writing this to a json file. Import the data into Escher by clicking Data $\rightarrow$ Load reaction data, then select your json file. Describe and discuss the difference in flux distributions.

In [160]:
# From: https://www.geeksforgeeks.org/how-to-convert-python-dictionary-to-json/ 
dic = {reac.id : flx.flux for reac,flx in zip(ecoli.reactions, ecoli.reactions)}

with open("json_ecoli.json", "w") as outfile:
    json.dump(dic, outfile)


(v) Setting the maximal substrate uptake flux to 10 mmol gDW<sup>-1</sup> h<sup>-1</sup>, maximize growth using each of the carbon sources listed in Table 1 individually under both aerobic and anaerobic conditions.

### Table 1 
| Substrate | Exchange reaction ID |
| --- | --- |
| acetate | EX_ac_e |
| acetaldehyde | EX_acald_e |
| 2-oxoglutarate | EX_akg_e | 
| ethanol | EX_etoh_e |
| D-fructose | EX_fru_e |
| fumarate | EX_fum_e |
| D-glucose | EX_glc__D_e |
| L-glutamine | EX_gln_L_e |
| L-glutamate | EX_glu_L_e |
| D-lactate | EX_lac_D_e |
| L-malate | EX_mal_L_e |
| pyruvate | EX_pyr_e |
| succinate | EX_succ_e |

In [163]:
ecoli.reactions.get_by_id("EX_glc__D_e").lower_bound = 0 

reac_lst = ["EX_ac_e", "EX_acald_e", "EX_akg_e", "EX_etoh_e",
            "EX_fru_e","EX_fum_e","EX_glc__D_e","EX_gln_L_e",
            "EX_glu_L_e","EX_lac_D_e","EX_mal_L_e",
            "EX_pyr_e","EX_succ_e"]


aerob = []
ecoli.reactions.get_by_id("EX_o2_e").lower_bound = -1000
for reac_id in reac_lst:
    #print(reac_id)
    ecoli.reactions.get_by_id(reac_id).lower_bound = -20.0
    aerob.append(ecoli.optimize().objective_value)

    # Reset the lower bound for the tested substrate:
    ecoli.reactions.get_by_id(reac_id).lower_bound = 0


    
anaerob = []
ecoli.reactions.get_by_id("EX_o2_e").lower_bound = 0
for reac_id in reac_lst:
    ecoli.reactions.get_by_id(reac_id).lower_bound = -20.0
    anaerob.append(ecoli.optimize().objective_value)
    #print(ecoli.optimize().objective_value)

    # Reset the lower bound for the tested substrate:
    ecoli.reactions.get_by_id(reac_id).lower_bound = 0

print("-"*40)
for x,y in zip(aerob,anaerob):
    print(x, "\t\t", y)

----------------------------------------
0.3893130740792487 		 -0.1097694840834248
0.607269235614096 		 -9.162257051070531e-17
1.098169394603894 		 0.0
0.699552380605221 		 0.0
1.7905689707194787 		 0.5162537983863329
0.7864891875841895 		 -0.011644200337812658
1.79056897071948 		 0.5162537983863331
1.1636002256184361 		 -0.1097694840834248
1.2424715963193096 		 -0.1097694840834248
0.7402810052052707 		 -0.011644200337812683
0.7864891875841891 		 -0.011644200337812615
0.6221083142523831 		 0.06554228431717418
0.8401341768471444 		 -0.1097694840834248


C:\Users\Legion\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\Legion\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\Legion\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\Legion\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
 

In [162]:
for i in ecoli.reactions:
    if (str(i.id).startswith("EX")):
        print(i.id)
    # else:
    #     print(i.name)
    #print(type(i))

EX_ac_e
EX_acald_e
EX_akg_e
EX_co2_e
EX_etoh_e
EX_for_e
EX_fru_e
EX_fum_e
EX_glc__D_e
EX_gln_L_e
EX_glu_L_e
EX_h_e
EX_h2o_e
EX_lac_D_e
EX_mal_L_e
EX_nh4_e
EX_o2_e
EX_pi_e
EX_pyr_e
EX_succ_e
